### 0) 라이브러리 import

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

### 1) 데이터셋 import

In [39]:
df = pd.read_csv('./data/processed/df.csv')
# df

### 2) 불용어 import 및 데이터 확인

In [40]:
stopwords = pd.read_table('./data/stopwords.txt')
#['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
# stopwords

In [41]:
# stopwords.info()

### 3) OTK를 사용한 토큰화

In [42]:
okt = Okt()
okt.morphs('와 이런 것도 영화라고 차라리 뮤직비디오를 만드는 게 나을 뻔', stem = True)

['오다', '이렇다', '것', '도', '영화', '라고', '차라리', '뮤직비디오', '를', '만들다', '게', '나다', '뻔']

In [43]:
X_train = []
for sentence in tqdm(df['review'][:7465]): # 데이터의 80%를 train 데이터로 쪼갬
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_train.append(stopwords_removed_sentence)

100%|██████████| 7465/7465 [00:38<00:00, 192.31it/s]


In [44]:
# df['review'][:7466]

In [45]:
print(X_train[:3])

[['판타지', '를', '넘다', '영화', '역사', '에', '남', '을', '명작', '이다', '내', '가', '이', '걸', '왜', '극장', '에서', '못', '보다', '폰', '으로', '보다', '때', '의', '감동', '과는', '비교', '가', '안되다'], ['이집트', '여행', '하', '는', '느낌', '의', '영화'], ['목포', '연설', '장면', '은', '넋', '을', '잃다', '보다', '되다']]


In [46]:
X_test = []
for sentence in tqdm(df['review'][7465:]): # 데이터의 20%를 test 데이터로 쪼갬
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_test.append(stopwords_removed_sentence)

100%|██████████| 1866/1866 [00:06<00:00, 289.64it/s]


### 4) 정수 인코딩

In [47]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train) # 단어 집합이 생성되는 동시에 고유한 정수가 부여됨

In [48]:
# print(tokenizer.word_index) # 단어가 11,000 개가 넘게 존재하며, 부여된 고유한 정수를 확인할 수 있음

각 정수는 전체 훈련 데이터에서 등장 빈도수가 높은 순서대로 부여되었기 때문에, 높은 정수가 부여된 단어들은 등장 빈도수가 매우 낮다는 것을 의미함

여기서는 빈도수가 낮은 단어들은 자연어 처리에서 배제하고자 함

등장 빈도수가 3회 미만인 단어들이 이 데이터에서 얼만큼의 비중을 차지하는지 확인해볼 것임

In [49]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 11410
등장 빈도가 2번 이하인 희귀 단어의 수: 7073
단어 집합에서 희귀 단어의 비율: 61.98948290972831
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 6.065065251784148


빈도수 2이하인 단어를 제거한 후의 단어집합의 크기

In [50]:
# 전체 단어 개수 중 빈도수 2이하인 단어는 제거.
# 0번 패딩 토큰을 고려하여 + 1
vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 4338


단어 집합의 크기는 4,338개

이를 케라스 토크나이저의 인자로 넘겨주고 텍스트 시퀀스를 정수 시퀀스로 변환함

In [51]:
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [52]:
print(X_train[:3]) # X_train 정수인코딩 확인용

[[604, 10, 165, 3, 624, 6, 330, 7, 170, 16, 63, 8, 1, 133, 72, 397, 32, 57, 4, 22, 4, 69, 5, 43, 665, 737, 8, 132], [2885, 1247, 158, 13, 64, 5, 3], [2529, 89, 11, 7, 894, 4, 21]]


In [53]:
print(X_test[:3]) # X_test 정수인코딩 확인용

[[1, 53, 40, 25, 96, 3044, 218, 3, 2266, 96, 509, 3, 9, 25, 4, 148, 124, 40, 175, 49, 3631, 48, 4], [7, 1157, 204, 53, 21], [68, 30, 197, 55, 3132, 64, 31, 154, 3215, 174, 1030, 154, 8, 48, 235]]


각 샘플 내의 단어들은 각 단어에 대한 정수로 변환된 것을 확인할 수 있음

단어의 개수는 4,338개로 제한되었으므로 "0번 단어 ~ 4,338번 단어" 까지만 사용 중임

0번 단어는 패딩을 위한 토큰임을 상기하자

train_data에서 y_train과 y_test를 별도로 저장해줌

In [54]:
y_train = np.array(df['sentiment'][:7465])
y_test = np.array(df['sentiment'][7465:])

In [55]:
print(len(y_train),len(y_test)) # 길이를 보아하니 잘 나누어 진 것 같다.

7465 1866


### 5) 빈 샘플(empty samples) 제거

전체 데이터에서 빈도수가 낮은 단어가 삭제되었다는 것은 빈도수가 낮은 단어만으로 구성되었던 샘플들은 빈(empty) 샘플이 되었다는 것을 의미함

빈 샘플들은 어떤 레이블이 붙어있던 의미가 없으므로 빈 샘플들을 제거해주는 작업을 하겠슴

각 샘플들의 길이를 확인해서 길이가 0인 샘플들의 인덱스를 받아오겠다.

In [56]:
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1] # X_train에서 길이가 0인 샘플 index리스트

In [57]:
print(len(drop_train)) # 길이가 0인 샘플은 10개가 있다.

10


In [58]:
# 빈 샘플들을 제거
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
print(len(X_train))
print(len(y_train))

7455
7455


C:\Users\User\anaconda3\envs\Linux\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


7455 개로 샘플의 수가 10개 줄어든 것을 확인할 수 있다.

### 6) 패딩